- 데이터셋:
    * 총 운항 기록 24/07/01~ 24/12/31
    * 총 이상운항 기록 24/07/01 ~ 24/12/31
- 가설 : 각종 원인으로 인한 지연 발생 후 접속원인으로 발생하는 지연사고가 자주 발생한다.

검증 목표 : 총 운항기록중 시간대별로 여객기와 화물수송기를 분류, 최초지연과 그 이후 이어지는 지연사건이 몇건이나 되는지 확인하여, 지연사고 최초발생과 그 이후 생기는 연결/접속 지연사고의 관계성을 밝힘



In [44]:
## [1] 모듈 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

import utils

In [45]:
## 데이터 로딩및 확인
dataDF07 = pd.read_excel('./DATA/normal/항공기출도착현황240701.xlsx')

# 계획시간 별로 소트하여 지연 현황을 찾고, 해당 지연 원인을 abnormal 폴더에서 해당날짜와 같은 날짜에서 찾아오기
dataDF07.head(20)

c:\Users\kdt\anaconda3\envs\PY39\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,출/도착구분,날짜,항공사,편명,출발공항코드,출발공항명,도착공항코드,도착공항명,계획시간,예상시간,도착시간,구분,현황
0,도착,20240701,에어아시아버하드,AK1623,BKI,코타키나발루,ICN,인천,00:05,00:31,00:25,여객,지연
1,도착,20240701,아틀라스화물항공,5Y555,JFK,뉴욕,ICN,인천,00:05,00:20,00:12,화물,지연
2,도착,20240701,카고룩스 이탈리아,C87736,MXP,밀라노,ICN,인천,00:05,00:22,00:16,화물,지연
3,도착,20240701,폴라에어카고,PO213,CVG,신시내티,ICN,인천,00:35,21:35,:,화물,취소
4,도착,20240701,몽골항공,OM309,UBN,칭기즈 칸(신 울란바토르) 국제공항,ICN,인천,00:40,00:40,00:32,여객,도착
5,도착,20240701,내셔널 항공,N8871,HAN,노이바이(하노이),ICN,인천,00:40,08:00,08:08,기타,지연
6,도착,20240701,대한항공,KE854,PEK,베이징,ICN,인천,00:45,00:31,00:22,여객,도착
7,도착,20240701,에어서울,RS704,NRT,도쿄/나리타,ICN,인천,00:45,01:07,01:00,여객,지연
8,도착,20240701,아시아나항공,OZ954,HAN,노이바이(하노이),ICN,인천,00:55,00:45,00:40,화물,도착
9,도착,20240701,실크웨이웨스트항공,7L128,GYD,헤이다르 알리예프,ICN,인천,01:00,11:56,11:50,화물,지연


In [46]:
dataDF07.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18499 entries, 0 to 18498
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   출/도착구분  18499 non-null  object
 1   날짜      18499 non-null  int64 
 2   항공사     18498 non-null  object
 3   편명      18499 non-null  object
 4   출발공항코드  18499 non-null  object
 5   출발공항명   18499 non-null  object
 6   도착공항코드  18499 non-null  object
 7   도착공항명   18499 non-null  object
 8   계획시간    18499 non-null  object
 9   예상시간    18499 non-null  object
 10  도착시간    18499 non-null  object
 11  구분      18499 non-null  object
 12  현황      18498 non-null  object
dtypes: int64(1), object(12)
memory usage: 1.8+ MB


In [47]:
# 데이터 분류방법 / 조건문  / groupby
dataDF07 = dataDF07.sort_values(['날짜', '계획시간'])
dataDF07.head()
# dataDF07.loc[(dataDF07['현황']=='지연')]
dataDF07.loc[:,'현황']
(dataDF07['현황'] == '지연').idxmax()


0

In [48]:
dataDF07['현황'] == '지연'

0         True
1         True
2         True
3        False
4        False
         ...  
18494    False
18495    False
18496    False
18497    False
18498    False
Name: 현황, Length: 18499, dtype: bool

In [41]:
lateDF = pd.DataFrame(index=range(30))

In [42]:
def findlate(df, end_idx):
    # if end_idx == len(df.index):
    
    first_idx = (df.loc[end_idx:,'현황'] == '지연').idxmax()
    first_not_idx = (df.loc[first_idx:,'현황'] != '지연').idxmax()
    # print('first_idx = ', first_idx)
    # print('first_not_idx = ', first_not_idx)
    
    lateCount = 0 
    
    for n in np.arange(first_idx, first_not_idx):
        lateDate = df.loc[first_idx+n,'날짜']
        lateName = df.loc[first_idx+n,'편명']
        lateDay = pd.read_csv(f"./DATA/abnormal/filtered_data_{lateDate}.csv")
        lateReason = lateDay.loc[(lateDay['항공편명'] == lateName), '비정상원인']
        # print(lateReason, lateReason.dtype)
        if lateReason.str.contains('연결').any():         
            lateCount += 1
   
    if lateDate:    
        if (lateCount, lateDate) in lateDF.index and lateDate in lateDF.columns:
            lateDF.loc[lateCount, lateDate] += 1
        else:
            lateDF.loc[lateCount, lateDate] = 1
    
    # print('end_idx = ', first_not_idx)
    if first_not_idx >= len(dataDF07.index):
        print( lateDF )
        quit
    
    return findlate(df, first_not_idx)
        

In [43]:
findlate(dataDF07,0)

KeyError: 18510

In [ ]:
lateDF

,20240701,20240702,20240703,20240704,20240705,20240706,20240707,20240708,20240709,20240710,...,20240722,20240723,20240724,20240725,20240726,20240727,20240728,20240729,20240730,20240731
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
